In [9]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from PIL import Image
import os
import albumentations as A
import csv

train_data_dir = '.\\train\\train'
classes = ['Гароу', 'Генос', 'Сайтама', 'Соник', 'Татсумаки', 'Фубуки']


transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, ))
    ])

class ImageDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.images = []
        self.labels = []
        for idx, cls in enumerate(classes):
            cls_dir = os.path.join(data_dir, cls)
            for img_name in os.listdir(cls_dir):
                self.images.append(os.path.join(cls_dir, img_name))
                self.labels.append(idx)
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label
    
train_dataset = ImageDataset(train_data_dir, transform=transform)

train_loader =  DataLoader(train_dataset, batch_size=32, shuffle=True)


In [5]:
class Net(nn.Module):
    def __init__(self, classes):
        super(Net, self).__init__()
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        self.resnet.fc = nn.Identity()
        for param in self.resnet.parameters():
            param.requires_grad = True
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(self.resnet.layer4[2].conv3.out_channels, len(classes))

    def forward(self, x):
        x = self.resnet(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x


In [6]:
model = Net(classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for ep in range(num_epochs):
    model.train()
    loss = 0
    for img, label in train_loader:
        optimizer.zero_grad()
        out = model(img)
        losses = criterion(out, label)
        losses.backward()
        optimizer.step()
        loss += losses.item()
    print(f"Эпоха: {ep+1}, Потери: {loss/len(train_loader):.5f}")


Эпоха: 1, Потери: 1.51135
Эпоха: 2, Потери: 0.30359
Эпоха: 3, Потери: 0.17321
Эпоха: 4, Потери: 0.11787
Эпоха: 5, Потери: 0.03740
Эпоха: 6, Потери: 0.01767
Эпоха: 7, Потери: 0.05788
Эпоха: 8, Потери: 0.16332
Эпоха: 9, Потери: 0.19709
Эпоха: 10, Потери: 0.09322


In [8]:
model.eval()
preprocess = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

test_path = "./test/test"
preds = []
images = []

# Записываем результаты предсказаний
for image_path in os.listdir(test_path):
    image = Image.open(os.path.join(test_path, image_path)).convert('RGB')
    image_tensor = preprocess(image)
    image_tensor = image_tensor.unsqueeze(0)
    images.append(image_path)
    with torch.no_grad():
        output = model(image_tensor)
        _, predicted = torch.max(output, 1)
        preds.append(predicted.item())


labels = ['Гароу', 'Генос', 'Сайтама', 'Соник', 'Татсумаки', 'Фубуки']
with open('submissiom.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['id', 'path', 'class']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i in range(len(preds)):
        writer.writerow({'id': i, 'path': images[i], 'class': labels[preds[i]]})